In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon
import geopandas as gpd

#### ZHVI

In [ ]:
zhvi_zip_path = 'https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1678040670'
zhvi_zip = pd.read_csv(zhvi_zip_path)

In [ ]:
#phx_zhvi = zhvi_zip[(zhvi_zip.City=='Phoenix')&(zhvi_zip.State=='AZ')].copy()
#phx_zhvi.head(2)

#### ZORI

In [ ]:
zori_zip_path='https://files.zillowstatic.com/research/public_csvs/zori/Zip_zori_sm_month.csv?t=1678040670'
zori_zip = pd.read_csv(zori_zip_path)

In [ ]:
#phx_zori = zori_zip[(zori_zip.City=='Phoenix')&(zori_zip.State=='AZ')].copy()

In [ ]:
#phx_zori.head(2)

## Map prices

In [ ]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
geo = gpd.read_file('../data/geo/shapefiles/Phoenix_ZCTA.shp')
geo['joinid'] = geo['GEOID20'].astype(int)
geo.head(2)

In [ ]:
villages = gpd.read_file('../data/geo/shapefiles/Villages.shp')
villages.head(2)

In [ ]:
villages = villages.to_crs(4269)
villages.crs == geo.crs

In [ ]:
road = gpd.read_file('../data/geo/shapefiles/maricopa_primroads.geojson')

## map

In [ ]:
own_cost = pd.merge(geo,zhvi_zip,how='left',left_on='joinid',right_on='RegionName')

In [ ]:
date_cols = list(own_cost.columns[own_cost.columns.get_loc('2000-01-31'):])
cur_month = date_cols[-1]

In [ ]:
own_cost = own_cost[['geometry','joinid']+date_cols]

In [ ]:
#melted table for animation of multiple months (if interested)
own_cost_melt = pd.melt(own_cost,id_vars=['joinid','geometry'],value_vars=date_cols)

In [ ]:
#table for current month for visualization
own_cost_cur = own_cost[['geometry','joinid',cur_month]]

In [ ]:
own_cost_cur.describe()

In [ ]:
fig, ax = plt.subplots(1, figsize = (20,16))
ax.axis('off')
new_norm = plt.Normalize(vmin=250000,vmax=750000)
sm = plt.cm.ScalarMappable(cmap='Blues', norm=new_norm)
sm.set_array([])
own_cost_cur.plot(column = '2023-01-31', cmap='Blues', norm=new_norm, edgecolor="white", linewidth=0.4, ax=ax)
villages.plot(ax=ax,color='none',edgecolor='gray',linewidth=0.8)
phx.plot(ax=ax,color='none',edgecolor='gray',linewidth=0.6)
    road.plot(ax=ax,color='black',edgecolor='none',linewidth=0.5)
fig.colorbar(sm)
#ax.set_title(f'Median Price {month[:3]} 20{month[-2:]}', color = 'k',  fontsize = 15)
#             xy = (0.6,.05), xycoords = 'figure fraction', fontsize = 11, color = 'k')

In [ ]:
plt.savefig(f'output/test/{month}.jpg', dpi = 250) # saving each